This notebook is used to convert Potential Temperature to Conservative Temperature in our boundary conditions. It makes use of the MATLAB gsw library.

Also, converts Practical Salinity to Reference Salinity.

In [1]:
import numpy as np
import netCDF4 as nc

import os
import subprocess as sp
from salishsea_tools import (nc_tools, gsw_calls)
import datetime


# Plan
1. Copy existing boundary files into a new file. This file has Practical Salinity and Potential Temperature (I checked and POM uses Potential temperature). 
2. Load temperature and salinity from new file
3. Run matlab wrapper function from gsw_calls with loaded temperature and salinity.
5. Overwrite in temperature/salinity variable in the new netCDF file and update metadata.
6. Close new netcdf file.

### Question: 
Is it okay to assume $\delta S$ = 0? Remember, $S_A = S_{ref} + \delta S$ , where $S_A$ is absolute salinity and $S_{ref}$ is the reference salinity.

See this notebook: http://nbviewer.jupyter.org/urls/bitbucket.org/salishsea/analysis-nancy/raw/tip/notebooks/teos-10/What%20is%20Delta%20S%20on%20our%20open%20boundaries.ipynb

### Decision: 
We will use $\delta S = 0$, and so $S_A = S_{ref}$. The script I have written can do either $S_A$ or $S_{ref}$ with a flag. In order to be clear, whenever I create BCs with $\delta S = 0$, I use Reference Salinity in the metadata to remind us that we chose $\delta S =0 $. 


# West

In [2]:
! cp /data/nsoontie/MEOPAR/NEMO-forcing/open_boundaries/west/SalishSea2_Masson_corrected.nc \
 /data/nsoontie/MEOPAR/NEMO-forcing/open_boundaries/west/SalishSea_west_TEOS10.nc

F = nc.Dataset('/data/nsoontie/MEOPAR/NEMO-forcing/open_boundaries/west/SalishSea_west_TEOS10.nc', 'r+')

In [3]:
sal = F.variables['vosaline']
temp = F.variables['votemper']
dep = np.expand_dims(np.expand_dims(np.expand_dims(F.variables['deptht'][:],axis=0),axis=2),axis=3) \
         + np.zeros(sal.shape)
long = F.variables['nav_lon'][:] + np.zeros(sal[:].shape)
lat = F.variables['nav_lat'][:] + np.zeros(sal[:].shape)

In [4]:
p = gsw_calls.generic_gsw_caller('gsw_p_from_z.m',[-dep,lat])

In [5]:
sal_pract = np.copy(sal[:])
sal_ref = gsw_calls.generic_gsw_caller('gsw_SR_from_SP.m',[sal_pract,])

In [6]:
temp_pot = np.copy(temp[:])
temp_cons = gsw_calls.generic_gsw_caller('gsw_CT_from_pt.m', [sal_ref, temp_pot,])

Rewrite temp and sal as conservative temperature and absoluate salinity

In [7]:
sal[:] = sal_ref
temp[:] = temp_cons

Update metdata

In [8]:
nc_tools.show_variable_attrs(F, 'vosaline')

<class 'netCDF4._netCDF4.Variable'>
float32 vosaline(time_counter, deptht, yb, xbT)
    units: 1
    long_name: Practical Salinity
    coordinates: nav_lon nav_lat deptht time_counter
    grid: SalishSea
unlimited dimensions: time_counter
current shape = (52, 40, 1, 870)
filling on, default _FillValue of 9.969209968386869e+36 used



In [9]:
sal.setncatts({'units': 'g/kg',
               'long_name': 'Reference Salinity'})
#sal.units = 'g/kg'
#sal.long_name =  'Absolute Salinity'

In [10]:
nc_tools.show_variable_attrs(F, 'votemper')

<class 'netCDF4._netCDF4.Variable'>
float32 votemper(time_counter, deptht, yb, xbT)
    units: degC
    long_name: Temperature
    coordinates: nav_lon nav_lat deptht time_counter
    grid: SalishSea
unlimited dimensions: time_counter
current shape = (52, 40, 1, 870)
filling on, default _FillValue of 9.969209968386869e+36 used



In [11]:
temp.setncatts({'units': 'deg C',
                'long_name': 'Conservative Temperature'})
#temp.units = 'deg C'
#temp.long_name = 'Conservative Temperature'

In [12]:
nc_tools.show_dataset_attrs(F)

file format: NETCDF4
Conventions: CF-1.6
institution: Dept of Earth, Ocean & Atmospheric Sciences, University of British Columbia
NCO: 4.0.9
comment: 
Deep salinity (below 150m) increased by a weekly factor for a better match with observed climatologies.

title: 
Modified boundary conditions based on weekly climatolgy from Diane Masson

source: 
https://bitbucket.org/salishsea/tools/src/tip/I_ForcingFiles/OBC/Improved TS Climatology Western Boundary.ipynb
https://bitbucket.org/salishsea/analysys/src/tip/Nancy/strat/comparisons/JuandeFucaBCs.ipynb

references: 
https://bitbucket.org/salishsea/nemo-forcing/src/tip/open_boundaries/west/SalishSea2_Masson_corrected.nc

history: 
2014-01-03 13:15:17] Created netCDF4 zlib=True dataset.
Sat Jan  4 13:05:15 2014: ncks -4 -L1 -d time_counter,0,51 SalishSea_TS_Masson.nc SalishSea_Masson_trunc.nc
[2014-01-04 14:17] Formed weekly climatology from original SalishSea grided results
[2014-01-24 ] Removed Nan's
[2014-01-25 18:01 ] Corrected for partial

In [13]:
F.title = """
Weekly climatology at the Juan de Fuca Strait open boundary in TEOS-10 variables. 
"""
F.source = """
https://bitbucket.org/salishsea/tools/src/tip/I_ForcingFiles/OBC/Improved TS Climatology Western Boundary.ipynb
https://bitbucket.org/salishsea/analysis/src/tip/Nancy/strat/comparisons/JuandeFucaBCs.ipynb
https://bitbucket.org/salishsea/tools/I_ForcingFiles/OBC/Temperature to conservative temperature in boundary conditions.ipynb
"""

F.comment = """
Temperature and salinity are TEOS-10 variables: Conservative Temperature and Reference Salinity
""" 
F.references = """
https://bitbucket.org/salishsea/nemo-forcing/src/tip/open_boundaries/west/SalishSea_west_TEOS10.nc
"""
F.history = """
2014-01-03 13:15:17] Created netCDF4 zlib=True dataset.
Sat Jan  4 13:05:15 2014: ncks -4 -L1 -d time_counter,0,51 SalishSea_TS_Masson.nc SalishSea_Masson_trunc.nc
[2014-01-04 14:17] Formed weekly climatology from original SalishSea grided results
[2014-01-24 ] Removed Nan's
[2014-01-25 18:01 ] Corrected for partial cells
[2015-10-19 ] Applied correction to increase deep salinity
[{} ] Converted temperarure and salinity to Conservative Temperature and Reference Salinity
""".format(datetime.datetime.today().strftime('%Y-%m-%d'))

In [14]:
F.close()

This crashes! So I have written script that can be run from the command line. create_TEOS-10_BCs.py

It can be used like this:

~~~~
python create_TEOS-10_BCs.py input.nc output.nc title_string
~~~~
The string is the title to be set in the output metadata.

This still crashed in python 3.5 when copying an existing file to a new file. I switched to a python 2.7 environment and it worked. 

# How I used the script

In a python 2.7 environment

## West

~~~~
python create_TEOS-10_BCs.py /data/nsoontie/MEOPAR/NEMO-forcing/open_boundaries/west/SalishSea2_Masson_corrected.nc /data/nsoontie/MEOPAR/NEMO-forcing/open_boundaries/west/SalishSea_west_TEOS10.nc "Weekly climatology of Temperature and Salinity at the Juan de Fuca boundary in TEOS-10 variables - Conservative Temperature and Reference Salinity"
~~~~

## North

~~~~
python create_TEOS-10_BCs.py /data/nsoontie/MEOPAR/NEMO-forcing/open_boundaries/north/SalishSea2_North_tra.nc /data/nsoontie/MEOPAR/NEMO-forcing/open_boundaries/north/SalishSea_north_TEOS10.nc "Seasonal climatology of Temperature and Salinity at the Johnstone Strait boundary in TEOS-10 variables - Conservative Temperature and Reference Salinity"
~~~~